In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import itertools
import functools
import importlib
import os
from IPython.display import Image

In [3]:
# Regression:
from nl_03_filter_model_score_reg import filter_split_model_score

### This script runs the neutral loss scripts for MALDI IMS data
nl_03 Filters and searchs with DeepChem and SciKit machine learning models.


In [5]:
# Load X's if running machine learning
join_df_path = 'all_public_data_output_02.pickle' 
join_df = pd.read_pickle(join_df_path)  # 0.5 Gb
unique_hmdbs = list(join_df.hmdb_ids.unique())

mord_df = pd.read_pickle('mord_norm_hmdb_df.pickle') # 0.2 Gb
mord_df = mord_df[mord_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

bits_df = pd.read_pickle('bits_hmdb_df.pickle') # 0.8 Gb
bits_df = bits_df[bits_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

In [6]:
# Can use this to filter on inferred vaccuum maldi vs ap maldi
# ap 1921, vav 1228, all 3248
beta_METASPACE = pd.read_csv('Metaspace_beta_2020_Feb.tsv', sep='\t')
inf_source = beta_METASPACE[['datasetId', 'analyzer']].copy(deep=True)

all_ds = list(inf_source.datasetId)

inf_source = inf_source[(inf_source.analyzer == 'FTICR') |
                       (inf_source.analyzer == 'Orbitrap')].copy(deep=True)
ap_ds = list(inf_source[inf_source.analyzer == 'Orbitrap'].datasetId)
vac_ds = list(inf_source[inf_source.analyzer == 'FTICR'].datasetId)

In [7]:
# nl_03: Filtering and machine learning models
datasets = ['all_ds', 'ap_ds', 'vac_ds']
ds_dict = {'all_ds': all_ds, 'ap_ds': ap_ds, 'vac_ds': vac_ds}
dataset = [datasets[1]]
target = ['H2O'] # ['H2O']
polarity = [1] # {1:positive, -1:negative] 
fdrs = [0.20] # [0.20, 0.10, 0.05]
colocalizations = [0.50] # [0.00, 0.50, 0.75]
min_n_obs = [10] # [1, 4, 10]
one_id_only = [False] # [True, False]

# Outputs (y), and weights (w):
global_ys = ['n_loss_only_H2O', 'n_loss_wparent_H2O', 'n_loss_all_H2O']
y = ['n_loss_wparent_H2O']
w = ['weight']
w_norms = [False, '10_y_bins', 'n_obs', 'isobar',
          '10_y_bins_W_n_obs', '10_y_bins_W_isobar', 'n_obs_W_isobar',
          '10_y_bins_W_n_obs_W_isobar']
w_norm = ['n_obs']

# Class of model:
models = ['direct', 'ml', 'deepchem']
model = [models[1]]

# Split.  If true, any formula only appears in train/test/validate.
single_fold_group = True

filter_model_df = pd.DataFrame(columns=['target', 'polarity', 'fdr', 'coloc', 'min_n_obs'
                                       'model', 'submodel', 'params', 'one_id_only',
                                        'X', 'y', 'w'])
# Specific inputs (X) for each model:
if model[0] is 'direct':
    submodel = ['']
    params = ['']
    Xs = ['trues', 'falses', 'rando', 'H2O_Present', 'n_loss_wparent_H2O']
    X = Xs                           
elif model[0] is 'ml':
    submodels = ['random_forest', 'XGBoost', 'SVM']
    submodel = [submodels[1]]
    params = ['']
    Xs = ['mord_norm', 'bits'] 
    X = ['mord_norm']
elif model[0] is 'deepchem':
    submodels = [('GraphConvModel', 'GraphConv'), 
              ('WeaveModel', 'Weave'), 
              ('MPNNModel','Weave') ] 
    submodel = [submodels[0]]    
    params = ['']
    Xs = ['Molecule', 'Smiles']
    X = [Xs[0]]          
    
model_list = list(itertools.product(*[target, polarity, fdrs, colocalizations, min_n_obs,
                                      model, submodel, params, one_id_only,
                                      X, y, w, w_norm, dataset]))
print()
filter_param_df = pd.DataFrame(model_list, columns=['target', 'polarity', 'fdr', 'coloc', 'min_n_obs',
                                                    'model', 'submodel', 'params', 'one_id_only',
                                                    'X', 'y', 'w', 'w_norm', 'dataset'])

filter_param_df

,target,polarity,fdr,coloc,min_n_obs,model,submodel,params,one_id_only,X,y,w,w_norm,dataset
0,H2O,1,0.2,0.5,10,ml,XGBoost,,False,mord_norm,n_loss_wparent_H2O,weight,n_obs,ap_ds


In [8]:
model_results = filter_split_model_score(filter_param_df, join_df_path, mord_df, bits_df,
                                         single_fold_group, ds_dict[dataset[0]])

start 0
filtering 0
aggregating 0
checking aggregation: 
             hmdb_ids    y  n_obs
67226   HMDB0124825+K  0.0      1
46269   HMDB0053245+H  0.0      5
50901   HMDB0061607+H  0.0      1
20099  HMDB0031284+Na  0.0      2
65450   HMDB0115892+K  0.0      4
(79719, 7)
Printing histogram!
n_obs_plots/2020_02_17_18_52_40_412378
Printing histogram!
hist_plots/2020_02_17_18_52_40_872052
joining X 0
['hmdb_ids', 'formula', 'intensity_avg', 'intensity_nl', 'y', 'w', 'n_obs']
Printing intensities parent and NL!
int_plots/2020_02_17_18_52_44_099799
splitting 0
X_df_shapes:
(12771, 1826)
(5637, 1826)
Testing split, n overlap train/test: 
{'formula': 0, 'hmdb_ids': 0, 'y': 66, 'X': 32}
modeling 0
ml :  XGBoost
train_y: (12771,)
train_w: (12771,)
train_X: (12771, 1826)
Weighting:  n_obs
Printing regression actual versus predicted output!
model_plots/2020_02_17_18_56_44_364723
Elapsed time:


Executed without error

258.05321407318115


In [9]:
model_results.round(3)

,target,polarity,fdr,coloc,min_n_obs,model,submodel,params,one_id_only,X,...,dataset,rmse_train,rmse_test,r2_test,rmse_train_w,rmse_test_w,r2_test_w,n_test,plt_path,time
0,H2O,1,0.2,0.5,10,ml,XGBoost,,False,mord_norm,...,ap_ds,0.091,0.136,0.334,0.073,0.122,0.401,5637,model_plots/2020_02_17_18_56_44_364723,258


In [18]:
model_results.sort_values(by=['rmse_test_w']).round(3)

,target,polarity,fdr,coloc,min_n_obs,model,submodel,params,one_id_only,X,...,dataset,rmse_train,rmse_test,r2_test,rmse_train_w,rmse_test_w,r2_test_w,n_test,plt_path,time
3,H2O,1,0.1,0.5,10,ml,XGBoost,,False,mord_norm,...,all_ds,0.174,0.241,-0.504,0.074,0.261,-0.122,3188,model_plots/2020_02_17_09_40_38_542789,85
0,H2O,1,0.2,0.0,10,ml,XGBoost,,False,mord_norm,...,all_ds,0.217,0.249,-0.300,0.130,0.304,-0.000,8432,model_plots/2020_02_17_09_33_24_176842,274
1,H2O,1,0.2,0.5,10,ml,XGBoost,,False,mord_norm,...,all_ds,0.187,0.212,-0.514,0.121,0.363,-0.218,9125,model_plots/2020_02_17_09_37_38_109924,253
2,H2O,1,0.1,0.0,10,ml,XGBoost,,False,mord_norm,...,all_ds,0.199,0.218,-0.010,0.101,0.379,-1.550,2825,model_plots/2020_02_17_09_39_13_395619,95


In [17]:
# Regression results:
all_model_results = pd.read_pickle('all_model_results.pickle')
all_model_results = pd.concat([all_model_results, model_results])
all_model_results.to_pickle('all_model_results.pickle')
all_model_results.round(3)

/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,X,coloc,dataset,fdr,min_n_obs,model,n_test,one_id_only,params,plt_path,...,rmse_test,rmse_test_w,rmse_train,rmse_train_w,submodel,target,time,w,w_norm,y
0,bits,0.0,all_ds,0.2,1,ml,24270,False,,model_plots/2020_02_15_13_15_13_099915,...,0.241,NaN,0.172,NaN,random_forest,H2O,90,weight,False,n_loss_wparent_H2O
1,bits,0.0,all_ds,0.2,1,ml,25010,False,,model_plots/2020_02_15_13_16_41_301545,...,0.300,0.385,0.253,0.217,random_forest,H2O,88,weight,10_y_bins,n_loss_wparent_H2O
2,bits,0.0,all_ds,0.2,1,ml,24631,False,,model_plots/2020_02_15_13_18_14_488664,...,0.238,0.192,0.178,0.129,random_forest,H2O,93,weight,n_obs,n_loss_wparent_H2O
3,bits,0.0,all_ds,0.2,1,ml,22477,False,,model_plots/2020_02_15_13_20_13_789588,...,0.241,0.244,0.174,0.163,random_forest,H2O,119,weight,isobar,n_loss_wparent_H2O
4,bits,0.0,all_ds,0.2,1,ml,23379,False,,model_plots/2020_02_15_13_21_34_899161,...,0.282,0.263,0.237,0.192,random_forest,H2O,81,weight,10_y_bins_W_n_obs,n_loss_wparent_H2O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mord_norm,0.5,all_ds,0.1,10,ml,60,True,,model_plots/2020_02_16_16_16_52_703131,...,0.244,0.352,0.095,0.054,XGBoost,H2O,12,weight,10_y_bins_W_n_obs,n_loss_wparent_H2O
0,mord_norm,0.0,all_ds,0.2,10,ml,8432,False,,model_plots/2020_02_17_09_33_24_176842,...,0.249,0.304,0.217,0.130,XGBoost,H2O,274,weight,10_y_bins_W_isobar,n_loss_wparent_H2O
1,mord_norm,0.5,all_ds,0.2,10,ml,9125,False,,model_plots/2020_02_17_09_37_38_109924,...,0.212,0.363,0.187,0.121,XGBoost,H2O,253,weight,10_y_bins_W_isobar,n_loss_wparent_H2O
2,mord_norm,0.0,all_ds,0.1,10,ml,2825,False,,model_plots/2020_02_17_09_39_13_395619,...,0.218,0.379,0.199,0.101,XGBoost,H2O,95,weight,10_y_bins_W_isobar,n_loss_wparent_H2O


In [83]:
all_model_results.sort_values(by=['rmse_test']).round(3)

,X,coloc,dataset,fdr,min_n_obs,model,n_test,one_id_only,params,plt_path,...,rmse_test,rmse_test_w,rmse_train,rmse_train_w,submodel,target,time,w,w_norm,y
90,mord_norm,0.5,all_ds,0.2,10,ml,8742,False,,model_plots/2020_02_16_02_09_13_862559,...,0.132,0.090,0.114,0.108,XGBoost,H2O,234,weight,n_obs,n_loss_wparent_H2O
91,mord_norm,0.5,all_ds,0.2,10,ml,8728,False,,model_plots/2020_02_16_02_13_08_806221,...,0.144,0.165,0.115,0.101,XGBoost,H2O,234,weight,isobar,n_loss_wparent_H2O
94,mord_norm,0.5,all_ds,0.2,10,ml,8610,False,,model_plots/2020_02_16_02_25_15_974324,...,0.148,0.168,0.118,0.096,XGBoost,H2O,241,weight,n_obs_W_isobar,n_loss_wparent_H2O
190,mord_norm,0.5,all_ds,0.1,10,ml,2764,False,,model_plots/2020_02_16_05_56_22_195396,...,0.155,0.191,0.131,0.098,XGBoost,H2O,89,weight,n_obs_W_isobar,n_loss_wparent_H2O
182,bits,0.5,all_ds,0.1,10,ml,2221,False,,model_plots/2020_02_16_05_45_49_467006,...,0.156,0.164,0.140,0.116,XGBoost,H2O,49,weight,n_obs_W_isobar,n_loss_wparent_H2O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,bits,0.5,all_ds,0.1,1,ml,10565,False,,model_plots/2020_02_16_04_28_59_848406,...,0.396,0.335,0.386,0.248,XGBoost,H2O,145,weight,10_y_bins_W_isobar,n_loss_wparent_H2O
5,bits,0.0,all_ds,0.2,1,ml,22516,False,,model_plots/2020_02_15_15_11_35_631823,...,0.401,0.318,0.398,0.262,XGBoost,H2O,320,weight,10_y_bins_W_isobar,n_loss_wparent_H2O
49,bits,0.0,all_ds,0.1,1,ml,519,True,,model_plots/2020_02_16_16_04_11_474918,...,0.412,0.380,0.342,0.167,XGBoost,H2O,17,weight,10_y_bins,n_loss_wparent_H2O
73,bits,0.5,all_ds,0.1,1,ml,511,True,,model_plots/2020_02_16_16_10_52_735309,...,0.412,0.364,0.360,0.162,XGBoost,H2O,17,weight,10_y_bins,n_loss_wparent_H2O
